In [1]:
%pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124 --force-reinstall

Looking in indexes: https://download.pytorch.org/whl/cu124
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.5 GB 796.8 kB/s eta 0:52:31
     ---------------------------------------- 0.0/2.5 GB 796.8 kB/s eta 0:52:31
     ---------------------------------------- 0.0/2.5 GB 744.7 kB/s eta 0:56:11
     ---------------------------------------- 0.0/2.5 GB 824.4 kB/s eta 0:50:45
     ---------------------------------------- 0.0/2.5 GB 849.0 kB/s eta 0:49:16
     ---------------------------------------- 0.0/2.5 GB 855.6 kB/s eta 0:48:53
     ---------------------------------------- 0.0/2.5 GB 898.8 kB/s eta 0:46:32
     ---------------------------------------- 0.0/2.5 GB 898.8 kB/s eta 0:46:32
     ---------------------------------------- 0.0/2.5 GB 924.4 kB/s eta 0:45:14
     ---

In [2]:
import torch

def format_pytorch_version(version_str):
    # Example input: "2.0.1+cu118" -> returns "2.0.1"
    return version_str.split('+')[0]

def format_cuda_version(cuda_str):
    # If CUDA is None (CPU-only PyTorch), return "cpu"
    if cuda_str is None:
        return "cpu"
    # Example: "11.8" -> "cu118"
    return "cu" + cuda_str.replace('.', '')

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)
CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

print(f"Installing packages for torch-{TORCH}+{CUDA}...")

%pip install --upgrade --no-cache-dir pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html

%pip install torch_geometric

Installing packages for torch-2.5.1+cu124...
Looking in links: https://data.pyg.org/whl/torch-2.5.1+cu124.html
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
     ------------ --------------------------- 0.5/1.7 MB 2.1 MB/s eta 0:00:01
     ------------------ --------------------- 0.8/1.7 MB 1.1 MB/s eta 0:00:01
     ------------------------ --------------- 1.0/1.7 MB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 1.7/1.7 MB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
     ------ --------------------------------- 0.5/3.5 MB 2.8 MB/s eta 0:00:02
     ------------ --------------------------- 1.0/3.5 MB 2.5 MB/s eta 0:00:01
     ------------------ --------------------- 1.6/3.5 MB 2.5 MB/s eta 0:00:01
     --------------------- ------------------ 1.8/3.5 MB 2.6 MB/s eta 0:00:01
     --------------------------- ------------ 2.4/3.

In [4]:
%pip install scikit-learn imbalanced-learn xgboost

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.1 MB 2.8 MB/s eta 0:00:04
   --- ------------------------------------ 1.0/11.1 MB 2.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/11.1 MB 2.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/11.1 MB 2.7 MB/s eta 0:00:04
   ---- ----------------------------------- 1.3/11.1 MB 2.7 MB/s eta 0:00:04
   ----- ---------------------------------- 1.6/11.1 MB 1.2 MB/s eta 0:00:08
   ------ --------------------------------- 1.8/11.1 MB 1.2 MB/s eta 0:00:08
   ------ --------------------------------- 1.8/11.1 MB 1.2 MB/s eta 0:00:08
   ------- -------------------------------- 2.1/11.1 MB 1.1 MB/s eta 0:00:09
   ------- -------------------------------- 2.1/11.1 MB 1.1 MB/s eta 0:00:09
   -------- ----------------

In [6]:
%pip install optuna

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.1 MB ? eta -:--:--
   -------------- ------------------------- 0.8/2.1 MB 1.3 MB/s eta 0:00:02
   ------------------- -------------------- 1.0/2.1 MB 1.3 MB/s eta 0:00:01
   ----------------------------- ---------- 1.6/2.1 MB 1.6 MB/s eta 0:00:01
   ---------------------------------- ----- 1.8/2.1 MB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 2.1/2.1 MB 1.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


### Hyperparameter Optimization Citeseer
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "alpha": 0.5,
        "K": trial.suggest_int("K", 15, 25),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [128, 256]),
        "norm": None,
        "dropout": trial.suggest_float("dropout", 0, 0.5),
        "margin": 0.5,
        "lpl_weight": trial.suggest_float("lpl_weight", 0.01, 0.99),
        "ratio": trial.suggest_float("ratio", 0.05, 0.25),
        "pos_weight": 1,
        "aggregation": trial.suggest_categorical("aggregation", ['sum', 'mean']),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "citeseer_scar_learnable_params.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=250)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-25 09:41:25,789] A new study created in memory with name: no-name-ad88087d-d332-440f-8abf-479b3105686e


Running experiment with seed=1:
 - alpha=0.5, K=24, layers=2, hidden=128, out=128
 - norm=None, dropout=0.23571224368030846, margin=0.5, lpl_weight=0.6937850687450927
 - ratio=0.11062880068184706, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.0658, LPL: 1.5191, Contrastive: 0.0388
Epoch 50, Loss: 1.0540, LPL: 1.5191, Contrastive: 0.0001
Epoch 100, Loss: 15.0081, LPL: 21.6321, Contrastive: 0.0001
 - Metrics: Accuracy=0.9173, F1=0.7899, Recall=0.7375, Precision=0.8503
Running experiment with seed=2:
 - alpha=0.5, K=24, layers=2, hidden=128, out=128
 - norm=None, dropout=0.23571224368030846, margin=0.5, lpl_weight=0.6937850687450927
 - ratio=0.11062880068184706, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.0145, LPL: 1.4306, Contrastive: 0.0719
Epoch 50, Loss: 0.9926, LPL: 1.4306, Contrastive: 0.0002
Epoch 100, Loss: 15.0118, LPL: 21.6375, Contrastive: 0.0001
 - Metrics: Accuracy=0.9270, F1=0.8149, Recall=0.7632, Precision=0.8742
Running experiment

[I 2025-02-25 09:42:51,646] Trial 0 finished with value: 0.8101923947376501 and parameters: {'K': 24, 'layers': 2, 'hidden_channels': 128, 'out_channels': 128, 'dropout': 0.23571224368030846, 'lpl_weight': 0.6937850687450927, 'ratio': 0.11062880068184706, 'aggregation': 'sum'}. Best is trial 0 with value: 0.8101923947376501.


 - Metrics: Accuracy=0.9285, F1=0.8172, Recall=0.7589, Precision=0.8852
Done. Results written to citeseer_experimentations\citeseer_scar_learnable_params_2502094125.csv.
Average F1 over 5 seeds: 0.8102 ± 0.0108
Running experiment with seed=1:
 - alpha=0.5, K=19, layers=2, hidden=128, out=256
 - norm=None, dropout=0.35679518372805663, margin=0.5, lpl_weight=0.8702544111112751
 - ratio=0.18638172311715578, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.2681, LPL: 1.4547, Contrastive: 0.0163
Epoch 50, Loss: 1.2660, LPL: 1.4547, Contrastive: 0.0000
 - Metrics: Accuracy=0.8350, F1=0.6222, Recall=0.6448, Precision=0.6011
Running experiment with seed=2:
 - alpha=0.5, K=19, layers=2, hidden=128, out=256
 - norm=None, dropout=0.35679518372805663, margin=0.5, lpl_weight=0.8702544111112751
 - ratio=0.18638172311715578, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.1784, LPL: 1.3506, Contrastive: 0.0236
Epoch 50, Loss: 1.1754, LPL: 1.3506, Contrastive: 0.00

[I 2025-02-25 09:44:11,577] Trial 1 finished with value: 0.5894328969809854 and parameters: {'K': 19, 'layers': 2, 'hidden_channels': 128, 'out_channels': 256, 'dropout': 0.35679518372805663, 'lpl_weight': 0.8702544111112751, 'ratio': 0.18638172311715578, 'aggregation': 'mean'}. Best is trial 0 with value: 0.8101923947376501.


...

 - Metrics: Accuracy=0.9285, F1=0.8317, Recall=0.8388, Precision=0.8247
Done. Results written to citeseer_experimentations\citeseer_scar_learnable_params_2502143018.csv.
Average F1 over 5 seeds: 0.8270 ± 0.0064
Running experiment with seed=1:
 - alpha=0.5, K=16, layers=1, hidden=256, out=256
 - norm=None, dropout=0.03396997178870443, margin=0.5, lpl_weight=0.9876969365996106
 - ratio=0.2071810203493858, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.2474, LPL: 1.2624, Contrastive: 0.0398
 - Metrics: Accuracy=0.9213, F1=0.8112, Recall=0.8031, Precision=0.8195
Running experiment with seed=2:
 - alpha=0.5, K=16, layers=1, hidden=256, out=256
 - norm=None, dropout=0.03396997178870443, margin=0.5, lpl_weight=0.9876969365996106
 - ratio=0.2071810203493858, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.1980, LPL: 1.2124, Contrastive: 0.0488
 - Metrics: Accuracy=0.9270, F1=0.8280, Recall=0.8345, Precision=0.8216
Running experiment with seed=3:
 - alpha=0.

[I 2025-02-25 14:32:04,661] Trial 248 finished with value: 0.8234907964746787 and parameters: {'K': 16, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'dropout': 0.03396997178870443, 'lpl_weight': 0.9876969365996106, 'ratio': 0.2071810203493858, 'aggregation': 'sum'}. Best is trial 211 with value: 0.835605441126372.


 - Metrics: Accuracy=0.9258, F1=0.8262, Recall=0.8374, Precision=0.8153
Done. Results written to citeseer_experimentations\citeseer_scar_learnable_params_2502143111.csv.
Average F1 over 5 seeds: 0.8235 ± 0.0063
Running experiment with seed=1:
 - alpha=0.5, K=16, layers=1, hidden=256, out=256
 - norm=None, dropout=0.05746461099351121, margin=0.5, lpl_weight=0.9674761466156178
 - ratio=0.21096923137501697, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.2258, LPL: 1.2657, Contrastive: 0.0381
 - Metrics: Accuracy=0.9222, F1=0.8138, Recall=0.8074, Precision=0.8203
Running experiment with seed=2:
 - alpha=0.5, K=16, layers=1, hidden=256, out=256
 - norm=None, dropout=0.05746461099351121, margin=0.5, lpl_weight=0.9674761466156178
 - ratio=0.21096923137501697, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.1578, LPL: 1.1952, Contrastive: 0.0463
 - Metrics: Accuracy=0.9276, F1=0.8316, Recall=0.8488, Precision=0.8151
Running experiment with seed=3:
 - alpha=

[I 2025-02-25 14:32:57,548] Trial 249 finished with value: 0.8252694279533801 and parameters: {'K': 16, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'dropout': 0.05746461099351121, 'lpl_weight': 0.9674761466156178, 'ratio': 0.21096923137501697, 'aggregation': 'sum'}. Best is trial 211 with value: 0.835605441126372.


 - Metrics: Accuracy=0.9255, F1=0.8246, Recall=0.8317, Precision=0.8177
Done. Results written to citeseer_experimentations\citeseer_scar_learnable_params_2502143204.csv.
Average F1 over 5 seeds: 0.8253 ± 0.0073
Best trial:
  Average F1: 0.835605441126372
  Best parameters:
    K: 16
    layers: 1
    hidden_channels: 256
    out_channels: 256
    dropout: 0.08036582403633102
    lpl_weight: 0.8704897384874204
    ratio: 0.19913437554534513
    aggregation: sum


### Hyperparameter Optimization Citeseer
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "citeseer",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": 0.5,
        "K": trial.suggest_int("K", 15, 25),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [128, 256]),
        "norm": None,
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": 0.5,
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.05, 0.25),
        "pos_weight": 1,
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "citeseer_sar_learnable_params.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=250)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-25 14:32:57,648] A new study created in memory with name: no-name-1ddade82-00f7-415f-bdcf-cb46c14bcf29


Running experiment with seed=1:
 - alpha=0.5, K=25, layers=2, hidden=256, out=128
 - norm=None, dropout=0.23522024944013312, margin=0.5, lpl_weight=0.974899320327853
 - ratio=0.06744147417045919, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.4624, LPL: 1.4986, Contrastive: 0.0574
Epoch 50, Loss: 1.4610, LPL: 1.4986, Contrastive: 0.0001
 - Metrics: Accuracy=0.9083, F1=0.7417, Recall=0.6248, Precision=0.9125
Running experiment with seed=2:
 - alpha=0.5, K=25, layers=2, hidden=256, out=128
 - norm=None, dropout=0.23522024944013312, margin=0.5, lpl_weight=0.974899320327853
 - ratio=0.06744147417045919, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.4089, LPL: 1.4432, Contrastive: 0.0746
Epoch 50, Loss: 1.4070, LPL: 1.4432, Contrastive: 0.0001
 - Metrics: Accuracy=0.9059, F1=0.7345, Recall=0.6177, Precision=0.9059
Running experiment with seed=3:
 - alpha=0.5, K=25, layers=2, hidden=256, out=128
 - norm=None, dropout=0.23522024944013312, margin=0.5, l

[I 2025-02-25 14:34:25,813] Trial 0 finished with value: 0.7311878110033405 and parameters: {'K': 25, 'layers': 2, 'hidden_channels': 256, 'out_channels': 128, 'dropout': 0.23522024944013312, 'lpl_weight': 0.974899320327853, 'ratio': 0.06744147417045919, 'aggregation': 'mean'}. Best is trial 0 with value: 0.7311878110033405.


 - Metrics: Accuracy=0.9089, F1=0.7421, Recall=0.6220, Precision=0.9198
Done. Results written to citeseer_experimentations\citeseer_sar_learnable_params_2502143257.csv.
Average F1 over 5 seeds: 0.7312 ± 0.0155
Running experiment with seed=1:
 - alpha=0.5, K=20, layers=1, hidden=128, out=256
 - norm=None, dropout=0.12058702394771532, margin=0.5, lpl_weight=0.7473275459102271
 - ratio=0.16995449518467848, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.0217, LPL: 1.3484, Contrastive: 0.0554
Epoch 50, Loss: 1.0077, LPL: 1.3484, Contrastive: 0.0001
Epoch 100, Loss: 13.4780, LPL: 18.0349, Contrastive: 0.0001
 - Metrics: Accuracy=0.9273, F1=0.8223, Recall=0.7989, Precision=0.8472
Running experiment with seed=2:
 - alpha=0.5, K=20, layers=1, hidden=128, out=256
 - norm=None, dropout=0.12058702394771532, margin=0.5, lpl_weight=0.7473275459102271
 - ratio=0.16995449518467848, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.0404, LPL: 1.3740, Contrastive: 0.05

[I 2025-02-25 14:35:55,466] Trial 1 finished with value: 0.811875750009959 and parameters: {'K': 20, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'dropout': 0.12058702394771532, 'lpl_weight': 0.7473275459102271, 'ratio': 0.16995449518467848, 'aggregation': 'sum'}. Best is trial 1 with value: 0.811875750009959.


...

 - Metrics: Accuracy=0.9261, F1=0.8250, Recall=0.8274, Precision=0.8227
Done. Results written to citeseer_experimentations\citeseer_sar_learnable_params_2502205353.csv.
Average F1 over 5 seeds: 0.8236 ± 0.0025
Running experiment with seed=1:
 - alpha=0.5, K=20, layers=1, hidden=256, out=256
 - norm=None, dropout=0.17219788846963627, margin=0.5, lpl_weight=0.9674423849705379
 - ratio=0.17954312793956098, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.2752, LPL: 1.3163, Contrastive: 0.0538
 - Metrics: Accuracy=0.9339, F1=0.8408, Recall=0.8288, Precision=0.8532
Running experiment with seed=2:
 - alpha=0.5, K=20, layers=1, hidden=256, out=256
 - norm=None, dropout=0.17219788846963627, margin=0.5, lpl_weight=0.9674423849705379
 - ratio=0.17954312793956098, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.2541, LPL: 1.2941, Contrastive: 0.0665
Epoch 50, Loss: 1.2520, LPL: 1.2941, Contrastive: 0.0001
 - Metrics: Accuracy=0.9273, F1=0.8210, Recall=0.7917, Pr

[I 2025-02-25 20:56:08,060] Trial 248 finished with value: 0.8266822626148894 and parameters: {'K': 20, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'dropout': 0.17219788846963627, 'lpl_weight': 0.9674423849705379, 'ratio': 0.17954312793956098, 'aggregation': 'sum'}. Best is trial 205 with value: 0.8318308688180981.


 - Metrics: Accuracy=0.9288, F1=0.8286, Recall=0.8174, Precision=0.8402
Done. Results written to citeseer_experimentations\citeseer_sar_learnable_params_2502205454.csv.
Average F1 over 5 seeds: 0.8267 ± 0.0085
Running experiment with seed=1:
 - alpha=0.5, K=22, layers=1, hidden=256, out=128
 - norm=None, dropout=0.17240479214242987, margin=0.5, lpl_weight=0.987133853121539
 - ratio=0.20515910504232293, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.3564, LPL: 1.3734, Contrastive: 0.0512
Epoch 50, Loss: 1.3557, LPL: 1.3734, Contrastive: 0.0001
 - Metrics: Accuracy=0.9276, F1=0.8257, Recall=0.8146, Precision=0.8372
Running experiment with seed=2:
 - alpha=0.5, K=22, layers=1, hidden=256, out=128
 - norm=None, dropout=0.17240479214242987, margin=0.5, lpl_weight=0.987133853121539
 - ratio=0.20515910504232293, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.2656, LPL: 1.2813, Contrastive: 0.0611
 - Metrics: Accuracy=0.9182, F1=0.8035, Recall=0.7932, Prec

[I 2025-02-25 20:57:08,834] Trial 249 finished with value: 0.8112575280093577 and parameters: {'K': 22, 'layers': 1, 'hidden_channels': 256, 'out_channels': 128, 'dropout': 0.17240479214242987, 'lpl_weight': 0.987133853121539, 'ratio': 0.20515910504232293, 'aggregation': 'sum'}. Best is trial 205 with value: 0.8318308688180981.


 - Metrics: Accuracy=0.9255, F1=0.8229, Recall=0.8217, Precision=0.8240
Done. Results written to citeseer_experimentations\citeseer_sar_learnable_params_2502205608.csv.
Average F1 over 5 seeds: 0.8113 ± 0.0137
Best trial:
  Average F1: 0.8318308688180981
  Best parameters:
    K: 20
    layers: 1
    hidden_channels: 256
    out_channels: 256
    dropout: 0.15710913243299005
    lpl_weight: 0.9822721262276471
    ratio: 0.17392076895103395
    aggregation: sum


### Hyperparameter Optimization Cora
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "alpha": 0.5,
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [128, 256]),
        "norm": None,
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": 0.5,
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.05, 0.5),
        "pos_weight": 1,
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "cora_scar_learnable_params.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-25 20:57:08,873] A new study created in memory with name: no-name-d1ccad2e-610d-4fd9-9ae3-8d618d891d2f


Running experiment with seed=1:
 - alpha=0.5, K=28, layers=2, hidden=256, out=256
 - norm=None, dropout=0.4293590563537464, margin=0.5, lpl_weight=0.768300305459503
 - ratio=0.39757401434480705, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.1059, LPL: 1.4259, Contrastive: 0.0448
Epoch 50, Loss: 1.0955, LPL: 1.4259, Contrastive: 0.0001
Epoch 100, Loss: 18.6532, LPL: 24.2785, Contrastive: 0.0000
 - Metrics: Accuracy=0.8737, F1=0.7993, Recall=0.8325, Precision=0.7686
Running experiment with seed=2:
 - alpha=0.5, K=28, layers=2, hidden=256, out=256
 - norm=None, dropout=0.4293590563537464, margin=0.5, lpl_weight=0.768300305459503
 - ratio=0.39757401434480705, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.1101, LPL: 1.4255, Contrastive: 0.0644
Epoch 50, Loss: 1.0952, LPL: 1.4255, Contrastive: 0.0001
Epoch 100, Loss: 18.6597, LPL: 24.2870, Contrastive: 0.0000
 - Metrics: Accuracy=0.9021, F1=0.8383, Recall=0.8399, Precision=0.8368
Running experiment w

[I 2025-02-25 20:58:20,083] Trial 0 finished with value: 0.8163677647776983 and parameters: {'K': 28, 'layers': 2, 'hidden_channels': 256, 'out_channels': 256, 'dropout': 0.4293590563537464, 'lpl_weight': 0.768300305459503, 'ratio': 0.39757401434480705, 'aggregation': 'mean'}. Best is trial 0 with value: 0.8163677647776983.


 - Metrics: Accuracy=0.8815, F1=0.7977, Recall=0.7738, Precision=0.8231
Done. Results written to cora_experimentations\cora_scar_learnable_params_2502205708.csv.
Average F1 over 5 seeds: 0.8164 ± 0.0160
Running experiment with seed=1:
 - alpha=0.5, K=25, layers=2, hidden=256, out=256
 - norm=None, dropout=0.2276710613239632, margin=0.5, lpl_weight=0.7682612312285718
 - ratio=0.4028628720616924, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.0380, LPL: 1.3433, Contrastive: 0.0257
Epoch 50, Loss: 1.0320, LPL: 1.3433, Contrastive: 0.0001
Epoch 100, Loss: 16.6529, LPL: 21.6761, Contrastive: 0.0000
 - Metrics: Accuracy=0.8951, F1=0.8310, Recall=0.8533, Precision=0.8097
Running experiment with seed=2:
 - alpha=0.5, K=25, layers=2, hidden=256, out=256
 - norm=None, dropout=0.2276710613239632, margin=0.5, lpl_weight=0.7682612312285718
 - ratio=0.4028628720616924, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.0483, LPL: 1.3541, Contrastive: 0.0347
Epoch 

[I 2025-02-25 20:59:20,430] Trial 1 finished with value: 0.8315860503119094 and parameters: {'K': 25, 'layers': 2, 'hidden_channels': 256, 'out_channels': 256, 'dropout': 0.2276710613239632, 'lpl_weight': 0.7682612312285718, 'ratio': 0.4028628720616924, 'aggregation': 'mean'}. Best is trial 1 with value: 0.8315860503119094.


...

 - Metrics: Accuracy=0.9354, F1=0.8927, Recall=0.8900, Precision=0.8954
Done. Results written to cora_experimentations\cora_scar_learnable_params_2502225359.csv.
Average F1 over 5 seeds: 0.8882 ± 0.0071
Running experiment with seed=1:
 - alpha=0.5, K=34, layers=2, hidden=128, out=128
 - norm=None, dropout=0.26592160722133085, margin=0.5, lpl_weight=0.32278832461331564
 - ratio=0.2615712783441287, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.4430, LPL: 1.3403, Contrastive: 0.0153
Epoch 50, Loss: 0.4327, LPL: 1.3403, Contrastive: 0.0001
 - Metrics: Accuracy=0.8419, F1=0.7235, Recall=0.6846, Precision=0.7671
Running experiment with seed=2:
 - alpha=0.5, K=34, layers=2, hidden=128, out=128
 - norm=None, dropout=0.26592160722133085, margin=0.5, lpl_weight=0.32278832461331564
 - ratio=0.2615712783441287, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.4528, LPL: 1.3344, Contrastive: 0.0326
Epoch 50, Loss: 0.4308, LPL: 1.3344, Contrastive: 0.0001
Epoch

[I 2025-02-25 22:56:17,739] Trial 98 finished with value: 0.7417867085250476 and parameters: {'K': 34, 'layers': 2, 'hidden_channels': 128, 'out_channels': 128, 'dropout': 0.26592160722133085, 'lpl_weight': 0.32278832461331564, 'ratio': 0.2615712783441287, 'aggregation': 'mean'}. Best is trial 66 with value: 0.8945212842992796.


 - Metrics: Accuracy=0.8693, F1=0.7555, Recall=0.6687, Precision=0.8683
Done. Results written to cora_experimentations\cora_scar_learnable_params_2502225514.csv.
Average F1 over 5 seeds: 0.7418 ± 0.0213
Running experiment with seed=1:
 - alpha=0.5, K=35, layers=2, hidden=256, out=256
 - norm=None, dropout=0.47439539793845614, margin=0.5, lpl_weight=0.58822690213903
 - ratio=0.2160631963051784, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.7667, LPL: 1.2640, Contrastive: 0.0563
Epoch 50, Loss: 0.7435, LPL: 1.2640, Contrastive: 0.0001
Epoch 100, Loss: 16.1909, LPL: 27.5249, Contrastive: 0.0001
 - Metrics: Accuracy=0.9357, F1=0.8917, Recall=0.8753, Precision=0.9086
Running experiment with seed=2:
 - alpha=0.5, K=35, layers=2, hidden=256, out=256
 - norm=None, dropout=0.47439539793845614, margin=0.5, lpl_weight=0.58822690213903
 - ratio=0.2160631963051784, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.7773, LPL: 1.2634, Contrastive: 0.0829
Epoch 50, 

[I 2025-02-25 22:57:34,295] Trial 99 finished with value: 0.8864968238422584 and parameters: {'K': 35, 'layers': 2, 'hidden_channels': 256, 'out_channels': 256, 'dropout': 0.47439539793845614, 'lpl_weight': 0.58822690213903, 'ratio': 0.2160631963051784, 'aggregation': 'sum'}. Best is trial 66 with value: 0.8945212842992796.


 - Metrics: Accuracy=0.9295, F1=0.8817, Recall=0.8704, Precision=0.8934
Done. Results written to cora_experimentations\cora_scar_learnable_params_2502225617.csv.
Average F1 over 5 seeds: 0.8865 ± 0.0041
Best trial:
  Average F1: 0.8945212842992796
  Best parameters:
    K: 33
    layers: 1
    hidden_channels: 256
    out_channels: 256
    dropout: 0.4347842824281326
    lpl_weight: 0.16679894025052477
    ratio: 0.24148126447928195
    aggregation: sum


### Hyperparameter Optimization Cora
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "cora",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": 0.5,
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [128, 256]),
        "norm": None,
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": 0.5,
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.05, 0.5),
        "pos_weight": 1,
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "cora_sar_learnable_params.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-25 22:57:34,332] A new study created in memory with name: no-name-418c82fb-88c1-4c89-a998-ddfef6c23497


Running experiment with seed=1:
 - alpha=0.5, K=34, layers=1, hidden=128, out=256
 - norm=None, dropout=0.49055583386404333, margin=0.5, lpl_weight=0.849910877780626
 - ratio=0.39276826346984645, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.1812, LPL: 1.3833, Contrastive: 0.0367
Epoch 50, Loss: 1.1757, LPL: 1.3833, Contrastive: 0.0002
Epoch 100, Loss: 23.2682, LPL: 27.3772, Contrastive: 0.0000
 - Metrics: Accuracy=0.8874, F1=0.8345, Recall=0.9401, Precision=0.7502
Running experiment with seed=2:
 - alpha=0.5, K=34, layers=1, hidden=128, out=256
 - norm=None, dropout=0.49055583386404333, margin=0.5, lpl_weight=0.849910877780626
 - ratio=0.39276826346984645, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.2497, LPL: 1.4661, Contrastive: 0.0240
Epoch 50, Loss: 1.2461, LPL: 1.4661, Contrastive: 0.0001
Epoch 100, Loss: 23.2726, LPL: 27.3824, Contrastive: 0.0000
 - Metrics: Accuracy=0.8708, F1=0.8138, Recall=0.9352, Precision=0.7203
Running experiment w

[I 2025-02-25 22:58:44,379] Trial 0 finished with value: 0.8161186858496444 and parameters: {'K': 34, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'dropout': 0.49055583386404333, 'lpl_weight': 0.849910877780626, 'ratio': 0.39276826346984645, 'aggregation': 'sum'}. Best is trial 0 with value: 0.8161186858496444.


 - Metrics: Accuracy=0.8733, F1=0.8161, Recall=0.9303, Precision=0.7268
Done. Results written to cora_experimentations\cora_sar_learnable_params_2502225734.csv.
Average F1 over 5 seeds: 0.8161 ± 0.0100
Running experiment with seed=1:
 - alpha=0.5, K=33, layers=1, hidden=128, out=256
 - norm=None, dropout=0.4501901340387927, margin=0.5, lpl_weight=0.8489543245097706
 - ratio=0.3039978353757287, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.1177, LPL: 1.3089, Contrastive: 0.0432
Epoch 50, Loss: 1.1112, LPL: 1.3089, Contrastive: 0.0002
Epoch 100, Loss: 23.0888, LPL: 27.1968, Contrastive: 0.0000
 - Metrics: Accuracy=0.9125, F1=0.8624, Recall=0.9083, Precision=0.8210
Running experiment with seed=2:
 - alpha=0.5, K=33, layers=1, hidden=128, out=256
 - norm=None, dropout=0.4501901340387927, margin=0.5, lpl_weight=0.8489543245097706
 - ratio=0.3039978353757287, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 1.1972, LPL: 1.4045, Contrastive: 0.0321
Epoch 50,

[I 2025-02-25 22:59:54,014] Trial 1 finished with value: 0.8469580084601877 and parameters: {'K': 33, 'layers': 1, 'hidden_channels': 128, 'out_channels': 256, 'dropout': 0.4501901340387927, 'lpl_weight': 0.8489543245097706, 'ratio': 0.3039978353757287, 'aggregation': 'sum'}. Best is trial 1 with value: 0.8469580084601877.


...

 - Metrics: Accuracy=0.9369, F1=0.8970, Recall=0.9108, Precision=0.8837
Done. Results written to cora_experimentations\cora_sar_learnable_params_2602004534.csv.
Average F1 over 5 seeds: 0.8911 ± 0.0042
Running experiment with seed=1:
 - alpha=0.5, K=28, layers=2, hidden=256, out=256
 - norm=None, dropout=0.390287854449124, margin=0.5, lpl_weight=0.2174863901173385
 - ratio=0.2657194959435572, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.3018, LPL: 1.2039, Contrastive: 0.0511
Epoch 50, Loss: 0.2619, LPL: 1.2039, Contrastive: 0.0001
Epoch 100, Loss: 5.2438, LPL: 24.1106, Contrastive: 0.0001
 - Metrics: Accuracy=0.9365, F1=0.8937, Recall=0.8839, Precision=0.9038
Running experiment with seed=2:
 - alpha=0.5, K=28, layers=2, hidden=256, out=256
 - norm=None, dropout=0.390287854449124, margin=0.5, lpl_weight=0.2174863901173385
 - ratio=0.2657194959435572, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.3169, LPL: 1.2946, Contrastive: 0.0452
Epoch 50, Lo

[I 2025-02-26 00:47:58,645] Trial 98 finished with value: 0.8818389649608959 and parameters: {'K': 28, 'layers': 2, 'hidden_channels': 256, 'out_channels': 256, 'dropout': 0.390287854449124, 'lpl_weight': 0.2174863901173385, 'ratio': 0.2657194959435572, 'aggregation': 'sum'}. Best is trial 93 with value: 0.8936709270447185.


 - Metrics: Accuracy=0.9332, F1=0.8906, Recall=0.9010, Precision=0.8805
Done. Results written to cora_experimentations\cora_sar_learnable_params_2602004649.csv.
Average F1 over 5 seeds: 0.8818 ± 0.0114
Running experiment with seed=1:
 - alpha=0.5, K=28, layers=1, hidden=256, out=256
 - norm=None, dropout=0.4356067933402399, margin=0.5, lpl_weight=0.2489609832902156
 - ratio=0.2729534670003436, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.3381, LPL: 1.2118, Contrastive: 0.0484
Epoch 50, Loss: 0.3017, LPL: 1.2118, Contrastive: 0.0001
Epoch 100, Loss: 6.0027, LPL: 24.1110, Contrastive: 0.0000
 - Metrics: Accuracy=0.9346, F1=0.8911, Recall=0.8851, Precision=0.8971
Running experiment with seed=2:
 - alpha=0.5, K=28, layers=1, hidden=256, out=256
 - norm=None, dropout=0.4356067933402399, margin=0.5, lpl_weight=0.2489609832902156
 - ratio=0.2729534670003436, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.3562, LPL: 1.2977, Contrastive: 0.0441
Epoch 50, 

[I 2025-02-26 00:49:11,156] Trial 99 finished with value: 0.886563012960574 and parameters: {'K': 28, 'layers': 1, 'hidden_channels': 256, 'out_channels': 256, 'dropout': 0.4356067933402399, 'lpl_weight': 0.2489609832902156, 'ratio': 0.2729534670003436, 'aggregation': 'sum'}. Best is trial 93 with value: 0.8936709270447185.


 - Metrics: Accuracy=0.9317, F1=0.8881, Recall=0.8973, Precision=0.8790
Done. Results written to cora_experimentations\cora_sar_learnable_params_2602004758.csv.
Average F1 over 5 seeds: 0.8866 ± 0.0035
Best trial:
  Average F1: 0.8936709270447185
  Best parameters:
    K: 28
    layers: 1
    hidden_channels: 256
    out_channels: 256
    dropout: 0.39653137239313224
    lpl_weight: 0.4331932670956203
    ratio: 0.2534043950024391
    aggregation: sum


### Hyperparameter Optimization Pubmed
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "alpha": 0.5,
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": None,
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": 0.5,
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.05, 0.5),
        "pos_weight": 1,
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "pubmed_scar_learnable_params.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-26 00:49:11,242] A new study created in memory with name: no-name-2473cfee-190c-4dc4-a407-1d7b9532f745


Running experiment with seed=1:
 - alpha=0.5, K=33, layers=2, hidden=256, out=256
 - norm=None, dropout=0.21361411778685968, margin=0.5, lpl_weight=0.8618514201550006
 - ratio=0.4828967953143956, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.3576, LPL: 1.5750, Contrastive: 0.0012
Epoch 50, Loss: 1.3574, LPL: 1.5750, Contrastive: 0.0001
 - Metrics: Accuracy=0.8709, F1=0.8502, Recall=0.9173, Precision=0.7923
Running experiment with seed=2:
 - alpha=0.5, K=33, layers=2, hidden=256, out=256
 - norm=None, dropout=0.21361411778685968, margin=0.5, lpl_weight=0.8618514201550006
 - ratio=0.4828967953143956, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.3597, LPL: 1.5775, Contrastive: 0.0012
Epoch 50, Loss: 1.3596, LPL: 1.5775, Contrastive: 0.0001
 - Metrics: Accuracy=0.8704, F1=0.8498, Recall=0.9181, Precision=0.7909
Running experiment with seed=3:
 - alpha=0.5, K=33, layers=2, hidden=256, out=256
 - norm=None, dropout=0.21361411778685968, margin=0.5, l

[I 2025-02-26 00:52:29,186] Trial 0 finished with value: 0.8481934536321166 and parameters: {'K': 33, 'layers': 2, 'hidden_channels': 256, 'out_channels': 256, 'dropout': 0.21361411778685968, 'lpl_weight': 0.8618514201550006, 'ratio': 0.4828967953143956, 'aggregation': 'mean'}. Best is trial 0 with value: 0.8481934536321166.


 - Metrics: Accuracy=0.8674, F1=0.8453, Recall=0.9072, Precision=0.7914
Done. Results written to pubmed_experimentations\pubmed_scar_learnable_params_2602004911.csv.
Average F1 over 5 seeds: 0.8482 ± 0.0018
Running experiment with seed=1:
 - alpha=0.5, K=31, layers=2, hidden=256, out=128
 - norm=None, dropout=0.42697257905250074, margin=0.5, lpl_weight=0.24719332570138494
 - ratio=0.15590579305520336, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.3618, LPL: 1.4423, Contrastive: 0.0070
Epoch 50, Loss: 0.3566, LPL: 1.4423, Contrastive: 0.0002
Epoch 100, Loss: 6.1097, LPL: 24.7163, Contrastive: 0.0000
 - Metrics: Accuracy=0.8977, F1=0.8598, Recall=0.7851, Precision=0.9501
Running experiment with seed=2:
 - alpha=0.5, K=31, layers=2, hidden=256, out=128
 - norm=None, dropout=0.42697257905250074, margin=0.5, lpl_weight=0.24719332570138494
 - ratio=0.15590579305520336, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.3626, LPL: 1.4456, Contrastive: 0.00

[I 2025-02-26 00:55:58,040] Trial 1 finished with value: 0.8575166509278477 and parameters: {'K': 31, 'layers': 2, 'hidden_channels': 256, 'out_channels': 128, 'dropout': 0.42697257905250074, 'lpl_weight': 0.24719332570138494, 'ratio': 0.15590579305520336, 'aggregation': 'mean'}. Best is trial 1 with value: 0.8575166509278477.


...

 - Metrics: Accuracy=0.8762, F1=0.8449, Recall=0.8441, Precision=0.8458
Done. Results written to pubmed_experimentations\pubmed_scar_learnable_params_2602031436.csv.
Average F1 over 5 seeds: 0.8512 ± 0.0033
Running experiment with seed=1:
 - alpha=0.5, K=31, layers=1, hidden=256, out=64
 - norm=None, dropout=0.1661228412682496, margin=0.5, lpl_weight=0.36738953157803583
 - ratio=0.1721412646175986, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.5326, LPL: 1.4466, Contrastive: 0.0018
Epoch 50, Loss: 0.5316, LPL: 1.4466, Contrastive: 0.0002
Epoch 100, Loss: 9.1217, LPL: 24.8284, Contrastive: 0.0000
 - Metrics: Accuracy=0.9027, F1=0.8698, Recall=0.8135, Precision=0.9345
Running experiment with seed=2:
 - alpha=0.5, K=31, layers=1, hidden=256, out=64
 - norm=None, dropout=0.1661228412682496, margin=0.5, lpl_weight=0.36738953157803583
 - ratio=0.1721412646175986, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.5344, LPL: 1.4516, Contrastive: 0.0018
Epo

[I 2025-02-26 03:21:17,317] Trial 48 finished with value: 0.8674313609566632 and parameters: {'K': 31, 'layers': 1, 'hidden_channels': 256, 'out_channels': 64, 'dropout': 0.1661228412682496, 'lpl_weight': 0.36738953157803583, 'ratio': 0.1721412646175986, 'aggregation': 'mean'}. Best is trial 26 with value: 0.8820574096865187.


 - Metrics: Accuracy=0.9009, F1=0.8671, Recall=0.8091, Precision=0.9340
Done. Results written to pubmed_experimentations\pubmed_scar_learnable_params_2602031809.csv.
Average F1 over 5 seeds: 0.8674 ± 0.0013
Running experiment with seed=1:
 - alpha=0.5, K=25, layers=1, hidden=64, out=128
 - norm=None, dropout=0.18389439975874666, margin=0.5, lpl_weight=0.20588708410827844
 - ratio=0.10629840961154591, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.2872, LPL: 1.3943, Contrastive: 0.0002
Epoch 50, Loss: 0.2872, LPL: 1.3943, Contrastive: 0.0001
Epoch 100, Loss: 4.2255, LPL: 20.5232, Contrastive: 0.0000
 - Metrics: Accuracy=0.8786, F1=0.8267, Recall=0.7252, Precision=0.9613
Running experiment with seed=2:
 - alpha=0.5, K=25, layers=1, hidden=64, out=128
 - norm=None, dropout=0.18389439975874666, margin=0.5, lpl_weight=0.20588708410827844
 - ratio=0.10629840961154591, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.2879, LPL: 1.3974, Contrastive: 0.0002

[I 2025-02-26 03:23:53,908] Trial 49 finished with value: 0.8282743234762687 and parameters: {'K': 25, 'layers': 1, 'hidden_channels': 64, 'out_channels': 128, 'dropout': 0.18389439975874666, 'lpl_weight': 0.20588708410827844, 'ratio': 0.10629840961154591, 'aggregation': 'mean'}. Best is trial 26 with value: 0.8820574096865187.


 - Metrics: Accuracy=0.8782, F1=0.8258, Recall=0.7232, Precision=0.9625
Done. Results written to pubmed_experimentations\pubmed_scar_learnable_params_2602032117.csv.
Average F1 over 5 seeds: 0.8283 ± 0.0025
Best trial:
  Average F1: 0.8820574096865187
  Best parameters:
    K: 26
    layers: 1
    hidden_channels: 256
    out_channels: 128
    dropout: 0.1656850743902576
    lpl_weight: 0.4486853072680699
    ratio: 0.2286442078457228
    aggregation: mean


### Hyperparameter Optimization Pubmed
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "pubmed",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": 0.5,
        "K": trial.suggest_int("K", 25, 35),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [64, 128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [64, 128, 256]),
        "norm": None,
        "dropout": trial.suggest_float("dropout", 0.1, 0.5),
        "margin": 0.5,
        "lpl_weight": trial.suggest_float("lpl_weight", 0.1, 1.0),
        "ratio": trial.suggest_float("ratio", 0.05, 0.5),
        "pos_weight": 1,
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "pubmed_sar_learnable_params.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-26 03:23:53,936] A new study created in memory with name: no-name-7f4cbf8f-985f-4bdf-acf7-67c58479d826


Running experiment with seed=1:
 - alpha=0.5, K=29, layers=2, hidden=128, out=128
 - norm=None, dropout=0.13019271568601654, margin=0.5, lpl_weight=0.9617526580395241
 - ratio=0.4686918535680597, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.5174, LPL: 1.5777, Contrastive: 0.0002
Epoch 50, Loss: 1.5174, LPL: 1.5777, Contrastive: 0.0001
 - Metrics: Accuracy=0.8649, F1=0.8455, Recall=0.9258, Precision=0.7780
Running experiment with seed=2:
 - alpha=0.5, K=29, layers=2, hidden=128, out=128
 - norm=None, dropout=0.13019271568601654, margin=0.5, lpl_weight=0.9617526580395241
 - ratio=0.4686918535680597, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 1.4896, LPL: 1.5488, Contrastive: 0.0002
Epoch 50, Loss: 1.4896, LPL: 1.5488, Contrastive: 0.0001
 - Metrics: Accuracy=0.8669, F1=0.8460, Recall=0.9154, Precision=0.7864
Running experiment with seed=3:
 - alpha=0.5, K=29, layers=2, hidden=128, out=128
 - norm=None, dropout=0.13019271568601654, margin=0.5, l

[I 2025-02-26 03:26:12,321] Trial 0 finished with value: 0.8441785629649907 and parameters: {'K': 29, 'layers': 2, 'hidden_channels': 128, 'out_channels': 128, 'dropout': 0.13019271568601654, 'lpl_weight': 0.9617526580395241, 'ratio': 0.4686918535680597, 'aggregation': 'mean'}. Best is trial 0 with value: 0.8441785629649907.


 - Metrics: Accuracy=0.8643, F1=0.8438, Recall=0.9177, Precision=0.7810
Done. Results written to pubmed_experimentations\pubmed_sar_learnable_params_2602032353.csv.
Average F1 over 5 seeds: 0.8442 ± 0.0014
Running experiment with seed=1:
 - alpha=0.5, K=35, layers=1, hidden=256, out=128
 - norm=None, dropout=0.2998522859023272, margin=0.5, lpl_weight=0.5609886595645283
 - ratio=0.4704598795028684, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.9071, LPL: 1.6155, Contrastive: 0.0019
Epoch 50, Loss: 0.9064, LPL: 1.6155, Contrastive: 0.0001
Epoch 100, Loss: 14.6880, LPL: 26.1823, Contrastive: 0.0000
 - Metrics: Accuracy=0.8733, F1=0.8544, Recall=0.9305, Precision=0.7897
Running experiment with seed=2:
 - alpha=0.5, K=35, layers=1, hidden=256, out=128
 - norm=None, dropout=0.2998522859023272, margin=0.5, lpl_weight=0.5609886595645283
 - ratio=0.4704598795028684, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.8935, LPL: 1.5912, Contrastive: 0.0019
Epo

[I 2025-02-26 03:29:46,339] Trial 1 finished with value: 0.853918427311636 and parameters: {'K': 35, 'layers': 1, 'hidden_channels': 256, 'out_channels': 128, 'dropout': 0.2998522859023272, 'lpl_weight': 0.5609886595645283, 'ratio': 0.4704598795028684, 'aggregation': 'mean'}. Best is trial 1 with value: 0.853918427311636.


...

 - Metrics: Accuracy=0.9031, F1=0.8767, Recall=0.8629, Precision=0.8910
Done. Results written to pubmed_experimentations\pubmed_sar_learnable_params_2602075820.csv.
Average F1 over 5 seeds: 0.8782 ± 0.0032
Running experiment with seed=1:
 - alpha=0.5, K=25, layers=2, hidden=128, out=256
 - norm=None, dropout=0.13511458111879743, margin=0.5, lpl_weight=0.2231074780696927
 - ratio=0.2698667009750044, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.3213, LPL: 1.4392, Contrastive: 0.0003
Epoch 50, Loss: 0.3212, LPL: 1.4392, Contrastive: 0.0001
Epoch 100, Loss: 4.5760, LPL: 20.5103, Contrastive: 0.0000
 - Metrics: Accuracy=0.9000, F1=0.8747, Recall=0.8739, Precision=0.8755
Running experiment with seed=2:
 - alpha=0.5, K=25, layers=2, hidden=128, out=256
 - norm=None, dropout=0.13511458111879743, margin=0.5, lpl_weight=0.2231074780696927
 - ratio=0.2698667009750044, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.3172, LPL: 1.4205, Contrastive: 0.0003
Ep

[I 2025-02-26 08:05:25,112] Trial 98 finished with value: 0.8754223410387084 and parameters: {'K': 25, 'layers': 2, 'hidden_channels': 128, 'out_channels': 256, 'dropout': 0.13511458111879743, 'lpl_weight': 0.2231074780696927, 'ratio': 0.2698667009750044, 'aggregation': 'mean'}. Best is trial 83 with value: 0.8798047359394902.


 - Metrics: Accuracy=0.8978, F1=0.8722, Recall=0.8728, Precision=0.8715
Done. Results written to pubmed_experimentations\pubmed_sar_learnable_params_2602080152.csv.
Average F1 over 5 seeds: 0.8754 ± 0.0036
Running experiment with seed=1:
 - alpha=0.5, K=25, layers=2, hidden=128, out=256
 - norm=None, dropout=0.11095177279889956, margin=0.5, lpl_weight=0.3240380657010947
 - ratio=0.26709136300274616, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.4664, LPL: 1.4388, Contrastive: 0.0003
Epoch 50, Loss: 0.4663, LPL: 1.4388, Contrastive: 0.0001
Epoch 100, Loss: 6.6475, LPL: 20.5144, Contrastive: 0.0000
 - Metrics: Accuracy=0.9016, F1=0.8767, Recall=0.8757, Precision=0.8777
Running experiment with seed=2:
 - alpha=0.5, K=25, layers=2, hidden=128, out=256
 - norm=None, dropout=0.11095177279889956, margin=0.5, lpl_weight=0.3240380657010947
 - ratio=0.26709136300274616, pos_weight=1, aggregation=mean, treatment=removal
Epoch 0, Loss: 0.4593, LPL: 1.4168, Contrastive: 0.0003


[I 2025-02-26 08:08:59,181] Trial 99 finished with value: 0.8772450709674819 and parameters: {'K': 25, 'layers': 2, 'hidden_channels': 128, 'out_channels': 256, 'dropout': 0.11095177279889956, 'lpl_weight': 0.3240380657010947, 'ratio': 0.26709136300274616, 'aggregation': 'mean'}. Best is trial 83 with value: 0.8798047359394902.


 - Metrics: Accuracy=0.9014, F1=0.8762, Recall=0.8738, Precision=0.8787
Done. Results written to pubmed_experimentations\pubmed_sar_learnable_params_2602080525.csv.
Average F1 over 5 seeds: 0.8772 ± 0.0023
Best trial:
  Average F1: 0.8798047359394902
  Best parameters:
    K: 26
    layers: 2
    hidden_channels: 64
    out_channels: 64
    dropout: 0.1254611964387077
    lpl_weight: 0.24515372552362366
    ratio: 0.24282619680377082
    aggregation: mean


### Hyperparameter Optimization WikiCS
#### SCAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SCAR",
        "train_pct": 0.5,
        "alpha": 0.5,#trial.suggest_float("alpha", 0.05, 1.0),
        "K": trial.suggest_int("K", 25, 45),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [ 128, 256]),
        "norm": None,
        "dropout": trial.suggest_float("dropout", 0, 0.5),
        "margin": 0.5,
        "lpl_weight": trial.suggest_float("lpl_weight", 0.01, 1.0),
        "ratio": trial.suggest_float("ratio", 0.01, 0.5),
        "pos_weight": 1, #trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "wikics_scar_learnable_params.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=150)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-26 19:33:24,883] A new study created in memory with name: no-name-bfc62dc5-3268-4228-93a8-b5bc374253f5


Running experiment with seed=1:
 - alpha=0.5, K=40, layers=2, hidden=256, out=128
 - norm=None, dropout=0.4453, margin=0.5, lpl_weight=0.6237826273527834
 - ratio=0.09987236788097459, pos_weight=1, aggregation=sum, treatment=removal


ERROR:root:Anomaly detector error: ValueError('Input contains NaN.')


Epoch 0, Loss: nan, LPL: 19.9822, Contrastive: nan
 - Metrics: Accuracy=0.9468, F1=0.8750, Recall=0.8126, Precision=0.9478
Running experiment with seed=2:
 - alpha=0.5, K=40, layers=2, hidden=256, out=128
 - norm=None, dropout=0.4453, margin=0.5, lpl_weight=0.6237826273527834
 - ratio=0.09987236788097459, pos_weight=1, aggregation=sum, treatment=removal


ERROR:root:Anomaly detector error: ValueError('Input contains NaN.')


...

Epoch 0, Loss: nan, LPL: 19.9822, Contrastive: nan
 - Metrics: Accuracy=0.9468, F1=0.8750, Recall=0.8126, Precision=0.9478
Running experiment with seed=2:
 - alpha=0.5, K=40, layers=2, hidden=256, out=128
 - norm=None, dropout=0.4453, margin=0.5, lpl_weight=0.6237826273527834
 - ratio=0.09987236788097459, pos_weight=1, aggregation=sum, treatment=removal


ERROR:root:Anomaly detector error: ValueError('Input contains NaN.')


Epoch 0, Loss: nan, LPL: 19.9822, Contrastive: nan


[W 2025-02-26 19:34:24,415] Trial 1 failed with parameters: {} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_1007428\2505435844.py", line 34, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\train_NNIF_GNN.py", line 571, in run_nnif_gnn_experiment
    pnn_model.fit(features_np, y_labels)
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\NNIF.py", line 585, in fit
    scores = score_func(X_neg)
             ^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNIF-GNN\NNIF.py", line 337, in score_samples
    obs_in_node = np.nonzero(path_col)[0]
                  ^^^^^^^^^^^^^^^^^^^^


KeyboardInterrupt: 

### Hyperparameter Optimization WikiCS
#### SAR 

In [ ]:
from train_NNIF_GNN import run_nnif_gnn_experiment
import optuna
from typing import Dict, Any

def objective(trial: optuna.trial.Trial) -> float:
    """
    Objective function for hyperparameter optimization using Optuna.
    It builds a parameter dictionary, calls the experiment function, and
    returns the average F1 score.
    """
    params: Dict[str, Any] = {
        "dataset_name": "wiki-cs",      
        "mechanism": "SAR",
        "train_pct": 0.5,
        "alpha": 0.5,#trial.suggest_float("alpha", 0.05, 1.0),
        "K": trial.suggest_int("K", 25, 45),
        "layers": trial.suggest_int("layers", 1, 2),
        "hidden_channels": trial.suggest_categorical("hidden_channels", [128, 256]),
        "out_channels": trial.suggest_categorical("out_channels", [ 128, 256]),
        "norm": None,
        "dropout": trial.suggest_float("dropout", 0, 0.5),
        "margin": 0.5,
        "lpl_weight": trial.suggest_float("lpl_weight", 0.01, 1.0),
        "ratio": trial.suggest_float("ratio", 0.01, 0.5),
        "pos_weight": 1, #trial.suggest_float("pos_weight", 0.5, 2.0),
        "aggregation": trial.suggest_categorical("aggregation", ["sum", "mean"]),
        "treatment": 'removal', #trial.suggest_categorical("treatment", ["removal", "relabeling"]),
        "seeds": 5,
        "output_csv": "wikics_sar_learnable_params.csv"
    }
    
    # Call the experiment function with these parameters.
    # run_nnif_gnn_experiment returns (avg_f1, std_f1)
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
    
    # We aim to maximize F1 score.
    return avg_f1

# Create an Optuna study to maximize the F1 score.
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

# Print out the best hyperparameters and corresponding F1 score.
print("Best trial:")
trial = study.best_trial
print("  Average F1:", trial.value)
print("  Best parameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-02-22 02:47:28,675] A new study created in memory with name: no-name-f19039dc-b2d8-4648-8747-7fb17d61d8be


Running experiment with seed=1:
 - alpha=0.3550773461077843, K=28, layers=2, hidden=256, out=128
 - norm=layernorm, dropout=0.17407591637864495, margin=0.8374569479625883, lpl_weight=0.8173330453434202
 - ratio=0.3813444753483398, pos_weight=1, aggregation=sum, treatment=removal


ERROR:root:Anomaly detector error: ValueError('Input contains NaN.')


Epoch 0, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 50, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 100, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 150, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 200, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 250, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 300, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 350, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 400, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 450, Loss: nan, LPL: 1.3863, Contrastive: nan
 - Metrics: Accuracy=0.8799, F1=0.7801, Recall=0.9302, Precision=0.6717
Running experiment with seed=2:
 - alpha=0.3550773461077843, K=28, layers=2, hidden=256, out=128
 - norm=layernorm, dropout=0.17407591637864495, margin=0.8374569479625883, lpl_weight=0.8173330453434202
 - ratio=0.3813444753483398, pos_weight=1, aggregation=sum, treatment=removal


ERROR:root:Anomaly detector error: ValueError('Input contains NaN.')


...

 - Metrics: Accuracy=0.9450, F1=0.8864, Recall=0.9365, Precision=0.8414
Done. Results written to wiki-cs_experimentations\wikics_sar_2202035308.csv.
Average F1 over 5 seeds: 0.8825 ± 0.0031
Running experiment with seed=1:
 - alpha=0.5186537938015142, K=34, layers=2, hidden=64, out=256
 - norm=graphnorm, dropout=0.45413115096218115, margin=0.14513996325002754, lpl_weight=0.4768964848328673
 - ratio=0.13048294540682323, pos_weight=1, aggregation=mean, treatment=relabeling
Epoch 0, Loss: 0.8083, LPL: 1.3863, Contrastive: 0.2814
 - Metrics: Accuracy=0.9507, F1=0.8926, Recall=0.8947, Precision=0.8904
Running experiment with seed=2:
 - alpha=0.5186537938015142, K=34, layers=2, hidden=64, out=256
 - norm=graphnorm, dropout=0.45413115096218115, margin=0.14513996325002754, lpl_weight=0.4768964848328673
 - ratio=0.13048294540682323, pos_weight=1, aggregation=mean, treatment=relabeling
Epoch 0, Loss: 0.8083, LPL: 1.3863, Contrastive: 0.2814
 - Metrics: Accuracy=0.9505, F1=0.8922, Recall=0.8944, P

[I 2025-02-22 03:57:24,656] Trial 28 finished with value: 0.8930925339787749 and parameters: {'alpha': 0.5186537938015142, 'K': 34, 'layers': 2, 'hidden_channels': 64, 'out_channels': 256, 'norm': 'graphnorm', 'dropout': 0.45413115096218115, 'margin': 0.14513996325002754, 'lpl_weight': 0.4768964848328673, 'ratio': 0.13048294540682323, 'aggregation': 'mean', 'treatment': 'relabeling'}. Best is trial 9 with value: 0.8965259618976467.


 - Metrics: Accuracy=0.9512, F1=0.8937, Recall=0.8959, Precision=0.8915
Done. Results written to wiki-cs_experimentations\wikics_sar_2202035540.csv.
Average F1 over 5 seeds: 0.8931 ± 0.0016
Running experiment with seed=1:
 - alpha=0.3288732664619557, K=34, layers=2, hidden=64, out=64
 - norm=graphnorm, dropout=0.40592581280698875, margin=0.16108055330874504, lpl_weight=0.4642332714768575
 - ratio=0.24161947437424713, pos_weight=1, aggregation=sum, treatment=removal
Epoch 0, Loss: 0.7837, LPL: 1.3863, Contrastive: 0.2615
Epoch 50, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 100, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 150, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 200, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 250, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 300, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 350, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 400, Loss: nan, LPL: 1.3863, Contrastive: nan
Epoch 450, Loss: nan, LPL: 1.3863, Contrastive: nan


ERROR:root:Anomaly detector WeightedIsoForest error: ValueError('Input contains NaN.')
[W 2025-02-22 03:58:03,577] Trial 29 failed with parameters: {'alpha': 0.3288732664619557, 'K': 34, 'layers': 2, 'hidden_channels': 64, 'out_channels': 64, 'norm': 'graphnorm', 'dropout': 0.40592581280698875, 'margin': 0.16108055330874504, 'lpl_weight': 0.4642332714768575, 'ratio': 0.24161947437424713, 'aggregation': 'sum', 'treatment': 'removal'} because of the following error: NotFittedError('need to call fit or load_model beforehand').
Traceback (most recent call last):
  File "c:\Users\romai\Desktop\gnn\gnn_pu\.conda\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\romai\AppData\Local\Temp\ipykernel_220368\2657924698.py", line 34, in objective
    avg_f1, std_f1 = run_nnif_gnn_experiment(params)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\romai\Desktop\gnn\gnn_pu\NNI

NotFittedError: need to call fit or load_model beforehand